In [6]:
import plotly.express as px
import pandas as pd
import pymssql
import json

In [9]:
database = "arctic_analysts_capstone"
user = "arctic_analysts"
password  = "ThisPassw0rd!"
server = "gen10-data-fundamentals-22-02-sql-server.database.windows.net"

def get_data(n = False):
    conn = pymssql.connect(server, user, password, database)
    cursor = conn.cursor()
    
    query = f"SELECT DISTINCT Date FROM building_permits"

    available_years_df = pd.read_sql(query, conn)
    available_years_df.sort_values(by = 'Date', inplace = True)
    available_dates = available_years_df.Date.astype('str').tolist()
    
    picked_date = available_dates[n]
    columns_to_select = 'FIPS, County, NewUnits'
    query = f"SELECT {columns_to_select} FROM building_permits WHERE building_permits.Date = '{picked_date}'"
    bpm_df = pd.read_sql(query, conn)
    print(f'\nBefore Data Management: {available_years_df.shape[0]}')

    return bpm_df

In [10]:
n = 0
while n < 300:
    bpm_df = get_data(n)
    print(bpm_df['NewUnits'].sum())
    n += 1


Before Data Management: 267
76644

Before Data Management: 267
80596

Before Data Management: 267
101374

Before Data Management: 267
92755

Before Data Management: 267
98607

Before Data Management: 267
106154

Before Data Management: 267
87599

Before Data Management: 267
97303

Before Data Management: 267
88556

Before Data Management: 267
90866

Before Data Management: 267
85810

Before Data Management: 267
76558

Before Data Management: 267
87401

Before Data Management: 267
82449

Before Data Management: 267
104257

Before Data Management: 267
101457

Before Data Management: 267
109581

Before Data Management: 267
106810

Before Data Management: 267
98551

Before Data Management: 267
105289

Before Data Management: 267
87631

Before Data Management: 267
92710

Before Data Management: 267
82938

Before Data Management: 267
81522

Before Data Management: 267
84844

Before Data Management: 267
86815

Before Data Management: 267
100404

Before Data Management: 267
107816

Before Dat

OperationalError: (20009, b'DB-Lib error message 20009, severity 9:\nUnable to connect: Adaptive Server is unavailable or does not exist (gen10-data-fundamentals-22-02-sql-server.database.windows.net)\nNet-Lib error during Unknown error (10060)\nDB-Lib error message 20009, severity 9:\nUnable to connect: Adaptive Server is unavailable or does not exist (gen10-data-fundamentals-22-02-sql-server.database.windows.net)\nNet-Lib error during Unknown error (10060)\n')

In [4]:
bpm_df = bpm_df[["CountyFips", "County", "StateFips", "1_Unit"]].copy()

In [5]:
bpm_df

,CountyFips,County,StateFips,1_Unit
0,001,Autauga County ...,01,33
1,003,Baldwin County ...,01,275
2,005,Barbour County ...,01,2
3,007,Bibb County ...,01,0
4,009,Blount County ...,01,1
...,...,...,...,...
3020,037,Sweetwater County ...,56,2
3021,039,Teton County ...,56,16
3022,041,Uinta County ...,56,1
3023,043,Washakie County ...,56,0


In [21]:
bpm_df['locale_fips'] = bpm_df.StateFips + bpm_df.CountyFips
bpm_df

,CountyFips,County,StateFips,1_Unit,locale_fips
0,001,Autauga County ...,01,33,01001
1,003,Baldwin County ...,01,275,01003
2,005,Barbour County ...,01,2,01005
3,007,Bibb County ...,01,0,01007
4,009,Blount County ...,01,1,01009
...,...,...,...,...,...
3020,037,Sweetwater County ...,56,2,56037
3021,039,Teton County ...,56,16,56039
3022,041,Uinta County ...,56,1,56041
3023,043,Washakie County ...,56,0,56043


In [23]:
with open('2021_US_Counties_3.7.json') as file:
    counties = json.load(file)

In [25]:
fig = px.choropleth(bpm_df, geojson=counties, locations = 'locale_fips', color = '1_Unit',
                   color_continuous_scale = "rdylgn_r", scope = "usa")

In [27]:
with open("test_map.html", 'w') as mapfile:
    mapfile.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))
mapfile.close()